## Import Packages

In [1]:
!pip install gurobipy>=9.5.1

zsh:1: 9.5.1 not found


In [2]:
import gurobipy as gp
from gurobipy import GRB as GRB
import sys
sys.path.insert(0, "../utils")

import column_generation
from unavailability import optimal_group2_unavailability

## Column Generation

In [3]:
generic_columns = column_generation.generic_column_generation(5)
group2_feasible_combinations = column_generation.group_column_generation(1, optimal_group2_unavailability, generic_columns)

Total possible combinations: 1048576
Total feasible combinations: 45046
Group 1 Unavailability Mapping: {1: [], 2: [], 3: [], 4: [], 5: [2, 5, 6, 7, 9, 10, 11, 12, 13, 14, 18], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [1, 8, 16, 17, 19], 12: [], 13: [16, 19], 14: [0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19], 15: [2, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 18, 19], 16: [0, 2, 6, 16, 19], 17: [0, 2, 6, 16, 19], 18: [0, 2, 6, 16, 19], 19: [0, 2, 6, 7, 9], 20: [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19], 21: [0, 1, 2, 6, 7, 8, 9, 16, 17, 19], 22: [7, 9, 10], 23: [0, 3, 4, 7, 9, 10, 15], 24: [0, 3, 4, 7, 9, 10, 15], 25: [10], 26: [1, 8, 10, 16, 17, 19], 27: [10], 28: [8], 29: [0, 3, 4, 8, 15], 30: [8], 31: [8], 32: [8], 33: [8], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [2, 5, 6, 7, 9, 10, 11, 12, 13, 14, 18], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [1, 8, 16, 17, 19], 46: [], 47: [16], 48: [0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18], 49: [2, 5

## Optimization Model

In [4]:
# Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)
model.setParam('Seed', 435)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Set parameter Seed to value 435


In [5]:
# Sets
D = range(1,6)
T = range(1,171)
T_d = {d : (T[(d-1)*len(T)//len(D):(d)*len(T)//len(D)])for d in D}
C = range(1,6)
M = range(6,21)
I = (*C,*M)
H_t = {t : range(1,len(group2_feasible_combinations[t-1])) for t in T}

In [6]:
# penalties
# separated to make them easier to read
# taken from this graph: https://www.desmos.com/calculator/g5rhs6n2ke
penalties = [(0,0), (4,4), (8, 11.31), (12, 20.78), (16, 32), (34, 99.13)]
Q = range(1,len(penalties)+1)

In [7]:
# Parameters

# need to sort this one out with column generation

a = group2_feasible_combinations
f = [[sum(a[t-1][h-1]) // 6 for h in H_t[t]] for t in T]
n = 8 # Need to inpute
u = {q : penalties[q-1][1] for q in Q}
v = {q : penalties[q-1][0] for q in Q}
w = .001 # Need to inpute

In [8]:
indices = [(t, h) for t in T for h in H_t[t]]

In [9]:
# Decision Varbiables
alpha = model.addVars(indices, vtype=GRB.BINARY, name="alpha")
y = model.addVars(I, D, Q, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name = "y")

In [10]:
for i in I:
    for d in D:
        model.addSOS(GRB.SOS_TYPE2, [y[i,d,q] for q in Q])

In [11]:
# Objective Value
model.setObjective(sum(f[t-1][h-1]*alpha[t,h] for t in T for h in H_t[t]) - w * sum(u[q] * y[i,d,q] for i in I for d in D for q in Q),sense=GRB.MAXIMIZE)

In [12]:
# Constraints
model.addConstrs(sum(alpha[t,h] for h in H_t[t]) <= 1 for t in T)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) <= n for i in I for d in D)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) == sum(v[q]*y[i,d,q] for q in Q) for i in I for d in D)
model.addConstrs(sum(y[i,d,q] for q in Q) == 1 for i in I for d in D)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


In [13]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 470 rows, 3379476 columns and 68490164 nonzeros
Model fingerprint: 0x8b889939
Model has 100 SOS constraints
Variable types: 600 continuous, 3378876 integer (3378876 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 14 rows and 156 columns (presolve time = 6s) ...
Presolve removed 14 rows and 156 columns (presolve time = 11s) ...
Presolve removed 14 rows and 156 columns (presolve time = 15s) ...
Presolve removed 14 rows and 156 columns (presolve time = 20s) ...
Presolve removed 14 rows and 156 columns (presolve time = 25s) ...
Presolve removed 14 rows and 156 column

In [14]:
filename = "../data/optimal_groups_basic_model/group2_results.txt"
with open(filename, "w") as file:
  for t in T:
    for h in H_t[t]:
      if alpha[t,h].X == 1:
        file.write(f"{t} {a[t-1][h-1]}\n")
        print(t, a[t-1][h-1])


1 (0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0)
2 (0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1)
4 (0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0)
5 (0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1)
6 (0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0)
7 (1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1)
8 (0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1)
12 (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0)
13 (1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0)
16 (0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0)
22 (1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0)
23 (0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0)
25 (1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1)
27 (0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1)
28 (0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0)
29 (0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1

In [15]:
print(sum(alpha[1,h].X for h in H_t[1]))

1.0
